<a href="https://colab.research.google.com/github/nicocrozier/catsvsdogs/blob/main/detect_(cleaned).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai
!pip install fastai --upgrade
!pip install -Uqq fastbook
!pip install DuckDuckGoImages
!pip install -q jmd_imagescraper
#!pip install voila
#!juypter serverextension enable voila --sys-prefix


In [2]:

import fastai
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *
from jmd_imagescraper.core import * 
from pathlib import Path
from jmd_imagescraper.imagecleaner import *

In [3]:
animals=['cat','dog']
path = Path().cwd()/"animals" #assigns path to current working directory

In [4]:
#downloads 100 images of 'cats' and saves it in path/cat
duckduckgo_search(path,"cat","cats",max_results=100) 


Duckduckgo search: cats


[Path('/content/animals/cat/201_8a6c3ab3.jpg'),
 Path('/content/animals/cat/202_9c776839.jpg'),
 Path('/content/animals/cat/203_143e5272.jpg'),
 Path('/content/animals/cat/204_3365996a.jpg'),
 Path('/content/animals/cat/205_720b200f.jpg'),
 Path('/content/animals/cat/206_f6c6a2b5.jpg'),
 Path('/content/animals/cat/207_f84dd160.jpg'),
 Path('/content/animals/cat/208_7d05d733.jpg'),
 Path('/content/animals/cat/209_7091ba23.jpg'),
 Path('/content/animals/cat/210_3270ccd5.jpg'),
 Path('/content/animals/cat/211_46ea6c4e.jpg'),
 Path('/content/animals/cat/212_81de5688.jpg'),
 Path('/content/animals/cat/213_fbe35ffa.jpg'),
 Path('/content/animals/cat/214_cb95f512.jpg'),
 Path('/content/animals/cat/215_cb4d3476.jpg'),
 Path('/content/animals/cat/216_23e8c7ae.jpg'),
 Path('/content/animals/cat/217_e9489c1c.jpg'),
 Path('/content/animals/cat/218_d34b2a74.jpg'),
 Path('/content/animals/cat/219_71f0b8a9.jpg'),
 Path('/content/animals/cat/220_35df8af0.jpg'),
 Path('/content/animals/cat/221_069651e4

In [5]:
# downloads 100 images of 'dogs' and saves it in path/dog
duckduckgo_search(path, "dog", "dogs", max_results=100)

Duckduckgo search: dogs


[Path('/content/animals/dog/201_0e073b91.jpg'),
 Path('/content/animals/dog/202_890d9f8a.jpg'),
 Path('/content/animals/dog/203_1758ff6f.jpg'),
 Path('/content/animals/dog/204_8ce2308b.jpg'),
 Path('/content/animals/dog/205_c2cf46b1.jpg'),
 Path('/content/animals/dog/206_9360b9f2.jpg'),
 Path('/content/animals/dog/207_4c8ea1ee.jpg'),
 Path('/content/animals/dog/208_fa803e67.jpg'),
 Path('/content/animals/dog/209_e0c9e2a8.jpg'),
 Path('/content/animals/dog/210_815ac2c4.jpg'),
 Path('/content/animals/dog/211_1112cc0f.jpg'),
 Path('/content/animals/dog/212_2ced181b.jpg'),
 Path('/content/animals/dog/213_9f3158e6.jpg'),
 Path('/content/animals/dog/214_48441d50.jpg'),
 Path('/content/animals/dog/215_0b6c67eb.jpg'),
 Path('/content/animals/dog/216_54af8a8a.jpg'),
 Path('/content/animals/dog/217_ec9c19f3.jpg'),
 Path('/content/animals/dog/218_f77c56bc.jpg'),
 Path('/content/animals/dog/219_f8f6efcf.jpg'),
 Path('/content/animals/dog/220_e73f278f.jpg'),
 Path('/content/animals/dog/221_76fa156c

In [6]:
#display_image_cleaner(path)

In [9]:
fns = get_image_files(path)
#fns

In [10]:
# look for failed downloads and unlink them (remove) from folder

failed = verify_images(fns) # looks for files that are not images
failed

(#0) []

In [11]:
failed.map(Path.unlink); # unlinks the failed files

In [12]:
# dataloaders

animals=DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.3, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms()
)

DataLoaders is a class which provides batches of a few items at a time to the GPU

In [13]:
dls=animals.dataloaders(path)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1023: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:760.)
  ret = func(*args, **kwargs)


**Train the model**

Call for a learner that learns based on the input images in 4 different training iterations or epochs. 



In [14]:
learn = cnn_learner(dls, resnet18, metrics=error_rate) # training model
learn.fine_tune(4)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch,train_loss,valid_loss,error_rate,time
0,0.588505,0.068691,0.033333,00:06


epoch,train_loss,valid_loss,error_rate,time
0,0.132125,0.036195,0.011111,00:07
1,0.090670,0.023867,0.016667,00:07
2,0.077278,0.040351,0.016667,00:07
3,0.064359,0.039803,0.016667,00:07


In [15]:
# exports the trained model so we can use in production
learn.export()
path = Path()
path.ls(file_exts='.pkl')

learn_inf = load_learner(path/'export.pkl')



Create a Notebook App

In [26]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
out_pl.clear_output()
lbl_pred = widgets.Label()
lbl_pred.value = 'Please choose an image'
btn_run = widgets.Button(description='Classify')

Event handler, a function that will be called when pressed.

In [27]:
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_pl.clear_output()
  with out_pl: display(img.to_thumb(128,128))
  pred, pred_idx, probs = learn_inf.predict(img)
  lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

Create a VBox (vertical box)

In [28]:
VBox([widgets.Label('Select your cat or dog, then click classify'),
      btn_upload, btn_run, out_pl, lbl_pred])